<a href="https://colab.research.google.com/github/anugrahjosiahgupta/CurserInteraction/blob/main/SmartVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive


Mounted at /content/drive
 affidavit.pdf			   'real CV (1).gdoc'
'Colab Notebooks'		   'real CV.gdoc'
 datasets			   'Resume march 2025 (1).docx'
 IMG-20240908-WA0003.jpg	   'Resume march 2025.docx'
 josh.pdf			   'Resume march 2025.pdf'
'Josh resume.gdoc'		    Untitled
'Josh resume.pdf'		   'Untitled document.gdoc'
'My signature _240827_223308.jpg'


In [ ]:
!ls /content/drive/MyDrive/datasets
!ls /content/drive/MyDrive/datasets/processed


processed
final_model.h5	freshcucumber  freshtamto    rottenbanana    rottenpatato
freshapples	freshoranges   products      rottencucumber  rottentamto
freshbanana	freshpatato    rottenapples  rottenoranges


In [ ]:
!ls /content


drive  sample_data


In [ ]:
!rm config.yaml


rm: cannot remove 'config.yaml': No such file or directory


In [ ]:
with open("config.yaml", "w", encoding="utf-8") as f:
    f.write("""paths:
  freshness_train: "/content/drive/MyDrive/datasets/train"
  freshness_val: "/content/drive/MyDrive/datasets/test"
  object_train: "/content/drive/MyDrive/datasets/train"
  object_model: "/content/models/object_model.joblib"
  freshness_model: "/content/models/freshness_model.keras"
  processed_data: "/content/drive/MyDrive/datasets/processed"

logging:
  file: "/content/logs/pipeline_log.txt"
""")


In [ ]:
!python main.py

python3: can't open file '/content/main.py': [Errno 2] No such file or directory


In [ ]:
import os

# Check contents of the current working directory
os.listdir()


['.config', 'config.yaml', 'drive', 'sample_data']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Check if Google Drive is mounted
print(os.listdir("/content/drive/MyDrive"))

# Check inside your 'datasets' folder
print(os.listdir("/content/drive/MyDrive/datasets"))

# Check inside 'processed'
print(os.listdir("/content/drive/MyDrive/datasets/processed"))


['Resume march 2025.pdf', 'Josh resume.gdoc', 'My signature _240827_223308.jpg', 'IMG-20240908-WA0003.jpg', 'affidavit.pdf', 'Josh resume.pdf', 'josh.pdf', 'Resume march 2025 (1).docx', 'Resume march 2025.docx', 'Untitled document.gdoc', 'Colab Notebooks', 'datasets', 'Untitled', 'real CV (1).gdoc', 'real CV.gdoc']
['processed']
['freshapples', 'freshbanana', 'freshcucumber', 'freshtamto', 'products', 'freshpatato', 'freshoranges', 'rottenbanana', 'rottenapples', 'rottencucumber', 'rottenpatato', 'rottenoranges', 'rottentamto', 'final_model.h5']


In [ ]:
import os

folder_path = "/content/drive/MyDrive/datasets/processed"
print("Files inside processed folder:", os.listdir(folder_path))


Files inside processed folder: ['freshapples', 'freshbanana', 'freshcucumber', 'freshtamto', 'products', 'freshpatato', 'freshoranges', 'rottenbanana', 'rottenapples', 'rottencucumber', 'rottenpatato', 'rottenoranges', 'rottentamto', 'final_model.h5']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Path to your processed folder
data_dir = "/content/drive/MyDrive/datasets/processed"

# Image settings
img_height, img_width = 224, 224
batch_size = 32

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Save model
model.save("/content/drive/MyDrive/datasets/processed/final_model.h5")
print("✅ Training complete and model saved as 'final_model.h5'")


Found 4455 images belonging to 13 classes.
Found 1111 images belonging to 13 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 10/140 ━━━━━━━━━━━━━━━━━━━━ 29:19 14s/step - accuracy: 0.1366 - loss: 2.8201

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import random
import tensorflow as tf


In [ ]:
# Load your model
model_path = "/content/drive/MyDrive/datasets/processed/final_model.h5"
model = load_model(model_path)

# Load face detection model from OpenCV (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define product fun facts
fun_facts = {
    "apple": "Apples are 25% air, which is why they float!",
    "banana": "Bananas are berries, but strawberries are not!",
    "orange": "Oranges are actually a hybrid between tangerines and pomelos.",
    "cucumber": "Cucumbers are made up of 95% water!",
    "tomato": "Tomatoes are a fruit, not a vegetable!",
    "potato": "Potatoes were the first vegetable grown in space!"
    # Add more as necessary
}

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB (model expects RGB, OpenCV captures BGR)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Prepare image for prediction
    img = cv2.resize(rgb_frame, (224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize

    # Predict the product
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions, axis=1)[0]
    confidence = predictions[0][class_idx]
    product_name = list(train_generator.class_indices.keys())[class_idx]

    # Display the prediction (fresh/rotten)
    if "fresh" in product_name:
        label = f"{product_name} - Fresh!"
        freshness = confidence
    else:
        label = f"{product_name} - Rotten!"
        freshness = confidence

    # Display freshness percentage
    percentage = f"Freshness: {f'{freshness*100:.2f}'}%"

    # Draw label and percentage
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, percentage, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display fun fact
    fun_fact = fun_facts.get(product_name.split()[1], "No fun fact available.")
    cv2.putText(frame, f"Fun Fact: {fun_fact}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Detect faces in the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # If a face is detected, predict attractiveness and suggest a fruit
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]

            # Attractiveness (using a dummy prediction model)
            attractiveness = random.uniform(60, 100)  # Placeholder (can be replaced by a real model)
            fruit_suggestion = random.choice(["apple", "banana", "orange", "cucumber", "tomato", "potato"])
            cv2.putText(frame, f"Attractiveness: {attractiveness:.1f}%", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            cv2.putText(frame, f"Try eating: {fruit_suggestion.capitalize()}", (x, y+h+10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Show the frame
    cv2.imshow("Product Quality Detection", frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from google.colab.patches import cv2_imshow
import random
import time

# Load trained model
model = load_model("/content/drive/MyDrive/datasets/processed/final_model.h5")

# Class labels
class_labels = ['freshbanana', 'freshapples', 'freshtamto', 'freshpatato', 'freshoranges', 'freshokra', 'freshcucumber',
                'rottenbanana', 'rottenapples', 'rottentamto', 'rottenpatato', 'rottenoranges', 'rottencucumber']

# Fun facts
fun_facts = {
    'banana': "Bananas are berries, but strawberries are not!",
    'apple': "Apples float because 25% of their volume is air!",
    'tamto': "Tomatoes were once considered poisonous in Europe.",
    'patato': "Potatoes were the first vegetable grown in space!",
    'oranges': "Oranges were originally green in color!",
    'okra': "Okra seeds are sometimes roasted and used as a coffee substitute.",
    'cucumber': "Cucumbers are 96% water!"
}

# Fruit suggestions for faces
fruit_suggestions = ['banana', 'apple', 'tamto', 'patato', 'oranges', 'okra', 'cucumber']

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start camera
cap = cv2.VideoCapture(0)  # Use 0 for webcam

print("📷 Starting camera... show a product or your face!")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    orig = frame.copy()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    detected = False

    for (x, y, w, h) in faces:
        detected = True
        face = frame[y:y+h, x:x+w]
        attractiveness = random.randint(60, 100)
        suggestion = random.choice(fruit_suggestions)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, f"Attractive: {attractiveness}%", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        cv2.putText(frame, f"Try eating more {suggestion}!", (x, y + h + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    if not detected:
        # Assume product detection if no face
        img = cv2.resize(orig, (224, 224))
        img = img.astype("float") / 255.0
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)

        prediction = model.predict(img)[0]
        max_index = np.argmax(prediction)
        label = class_labels[max_index]
        confidence = round(prediction[max_index] * 100, 2)

        is_fresh = "fresh" in label
        fruit = ''.join([char for char in label if not char.isnumeric()]).replace("fresh", "").replace("rotten", "")
        fruit = fruit.strip()

        safe = "✅ Safe to eat" if is_fresh else "⚠️ Not safe"
        verdict = "Fresh" if is_fresh else "Rotten"

        # Draw rectangle and prediction
        cv2.rectangle(frame, (10, 30), (400, 140), (0, 0, 255), 2)
        cv2.putText(frame, f"{verdict} {fruit}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        cv2.putText(frame, f"Confidence: {confidence}%", (20, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, safe, (20, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if is_fresh else (0, 0, 255), 2)
        cv2.putText(frame, "Fact: " + fun_facts.get(fruit, "No fact found."), (20, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)

    cv2_imshow(frame)

    # Wait key for stop
    key = cv2.waitKey(1)
    if key == ord("q"):
        print("👋 Exiting...")
        break

cap.release()
cv2.destroyAllWindows()
